In [1]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import random
import pygame
from loguru import logger

# --- Constants ---
SCREEN_WIDTH = 1200
SCREEN_HEIGHT = 800
FPS = 60

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (220, 50, 50)
GREEN = (50, 220, 50)
BLUE = (50, 50, 220)
GRAY = (150, 150, 150)


In [2]:
import gymnasium as gym
from gymnasium.spaces import Box, Discrete,Tuple
import numpy as np
import pygame

# Define colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)  # Color for the trajectory

class CustomEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 60}

    def __init__(self,render_mode=None):
        super().__init__()
        self.grid_size = 10
        self.pause = False
        self.domain_randomization = False
        self.render_mode = render_mode

        # Define two separate thresholds for obstacle handling
        self.distance_threshold_penalty = 4  # Penalty zone threshold (larger value)
        self.distance_threshold_collision = 1.5  # Collision threshold (smaller value)
        self.penalty_factor = 8  # Penalty scaling factor
        self.distance_reward_factor = 10
        self.smooth_action_penalty = 0.5
        self.steps = 0
        self.margin = 0.1
        


        self.current_distance = 0  # Current distance to goal, used for reward shaping
        self.max_steps = 40  # Set a maximum number of steps to prevent infinite loops
        # Action space (dx, dy)
        self.action_space = Box(low=-1, high=1, shape=(2,), dtype=np.float32)
        # Observation space (robot_x, robot_y, goal_x, goal_y)
        self.observation_shape = 2+2+2+1+1+1+2  # Robot position, hand position, velocity_hand,radius_hand, and distance to hand
        self.observation_space = Box(low=0, high=np.array([self.grid_size*2,self.grid_size, self.grid_size*2, self.grid_size,1,1 , (2**0.5)*self.grid_size,0.5*self.grid_size,0.5*self.grid_size,2*self.grid_size,self.grid_size]), 
                                     shape=(self.observation_shape,), dtype=np.float32)

        self.random = True
        # For rendering
        self.window = None
        self.clock = None
        self.cell_size = 50 # Pixels per grid unit
        self.trajectory_points = [] # New: List to store past robot positions
        self.dist_arm = 0


    def dist_point_to_segment_correct(self,P, A, B, eps=1e-12):
        P = np.asarray(P, dtype=float)
        A = np.asarray(A, dtype=float)
        B = np.asarray(B, dtype=float)
        v = B - A
        w = P - A
        vv = np.dot(v, v)
        if vv <= eps:
            # A and B coincide: treat as point A
            C = A.copy()
            d = np.linalg.norm(P - A)
            t = 0.0
            case = 'endpoint_A'
        else:
            t = np.dot(w, v) / vv
            if t < 0.0:
                C = A
                d = np.linalg.norm(P - A)
                case = 'before_A'
            elif t > 1.0:
                C = B
                d = np.linalg.norm(P - B)
                case = 'after_B'
            else:
                C = A + t * v
                d = np.linalg.norm(P - C)
                case = 'on_segment'
        return float(d), C, float(t), case

    def _get_obs(self):
        
        return np.concatenate(([self.robot_position]+ [self.hand_position]+[self.last_action]+
                               [np.array([self.current_distance])]+
                               [np.array([min(self.robot_position[0],
                                              self.robot_position[1],
                                              self.grid_size-self.robot_position[0],
                                              self.grid_size-self.robot_position[1]) ])]+
                                              [np.array([self.dist_arm])]+
                                               [self.fixed_point]))

    def _get_info(self):
        return {
            "distance_to_hand": self.current_distance,
            "robot_position": self.robot_position,
            "hand_position": self.hand_position,
        }

    def reset(self, seed=None, options=None):

        super().reset()
        self.stride_hand = np.random.uniform(0.6, 1)  # Randomize stride length
        self.stride_robot = 1  # Randomize stride length
        self.distance_threshold_collision = np.random.uniform(1, 2)  # Randomize collision threshold
        self.distance_threshold_penalty = np.random.uniform(2, 5)  # Randomize penalty threshold
        
        
        self.noise_obs_sigma = np.random.uniform(0, 0.1)  # Add some noise to observation to make it more realistic
        self.noise_action_sigma = np.random.uniform(0,0.1)  # Add some noise to action to make it more realistic
        
        
        
        self.robot_position = np.random.uniform(self.margin, [2*(self.grid_size-self.margin),self.grid_size-self.margin], size=2)
        self.hand_position = np.random.uniform(self.margin, [2*(self.grid_size-self.margin),self.grid_size-self.margin], size=2)
        # self.hand_position = np.clip(self.hand_position, self.margin, self.grid_size-self.margin)  # Ensure hand stays within grid bounds
        
        self.hand_move_mode = 'random' if np.random.rand() < 0.1 else 'towards_robot'  # Randomize hand movement mode
        
        self.current_distance = np.linalg.norm(self.robot_position - self.hand_position)
        self.pre_distance = self.current_distance
        self.last_action = np.zeros(2)
        self.steps = 0
        self.trajectory_points = [self.robot_position.copy()] # New: Reset trajectory and add initial position
        
        self.fixed_point = np.array([self.grid_size*random.uniform(0.2,1.8),self.grid_size])
        return self._get_obs(), self._get_info()

    def _reward(self,action):
        terminated = False
        truncated = False
        reward = 0  # Initialize reward
        done_reason = None  # Initialize done reason

        # action regulation penalty

        reward -= 0.5 * np.sum(np.square(action))  # Penalty for large actions

        self.dist_arm = self.dist_point_to_segment_correct(self.robot_position,self.hand_position, self.fixed_point)[0]
        if self.dist_arm < 1.5:
            reward -= 100
            terminated = True  # Truncate if arm is too short


        if np.any(self.robot_position <= self.margin) or (self.grid_size-self.robot_position[1] <=self.margin) or (2*self.grid_size-self.robot_position[0] <=self.margin):
            reward -= 400
            terminated = True  # Truncate if robot goes out of bounds
            done_reason = "out of bounds"

        # self.robot_position = np.clip(self.robot_position, self.margin, self.grid_size-self.margin)  # Ensure robot stays within grid bounds
        # boundary penalty

        reward -= self.smooth_action_penalty*np.linalg.norm(action-self.last_action)  # Penalty for smooth action
        self.last_action = action
        # Auxiliary Rewards -  distance to hand
        self.current_distance = np.linalg.norm(self.robot_position - self.hand_position)
        reward += (self.current_distance-self.pre_distance)*self.distance_reward_factor  # Reward shaping based on distance change
        self.pre_distance = self.current_distance

        # Obstacle handling with two thresholds
        if self.current_distance < self.distance_threshold_collision:
            reward -= 100
            terminated = True  # Terminate if too close to obstacles
            done_reason = "collision with obstacle"
        elif self.current_distance < self.distance_threshold_penalty:
            reward -= self.penalty_factor * (self.distance_threshold_penalty - self.current_distance)  # Penalty for being too close to obstacles

        
        
        reward+=  4  # Small reward for each step taken to encourage exploration
        if self.steps >= self.max_steps:
            reward += 50
            truncated = True  # Truncate if max steps reached
        return reward,terminated,truncated,done_reason

    def _get_hand_movement(self):

        if self.hand_move_mode == 'random':
            move_hand = np.random.uniform(-1, 1, size=2)  # Randomly move the hand position slightly
        elif self.hand_move_mode == 'towards_robot':
            dir_vector = self.robot_position - self.hand_position
            if np.linalg.norm(dir_vector) > 0:
                dir_vector /= np.linalg.norm(dir_vector)
            move_hand = dir_vector * self.stride_hand  # Move hand towards robot position
        
        return move_hand






    def step(self, action):
        if self.random:
            action+=np.random.normal(0,self.noise_action_sigma,size=self.action_space.shape)  # Add some noise to action to make it more realistic

        move_hand = self._get_hand_movement()
        self.hand_position += move_hand  # Update hand position
        self.hand_position = np.clip(self.hand_position, self.margin, [self.grid_size*2,self.grid_size-self.margin])  # Ensure hand stays within grid bounds

        self.robot_position += action * self.stride_robot  # Scale the action to control speed
        self.trajectory_points.append(self.robot_position.copy()) # New: Add current position to trajectory
        self.steps += 1
        

        reward,terminated,truncated,done_reason = self._reward(action)
        info = self._get_info()
        info['done_reason'] = done_reason
        observation = self._get_obs()
        if self.random:
            observation += np.random.normal(0, self.noise_obs_sigma, size=self.observation_shape)  # Add some noise to observation to make it more realistic

        return observation, reward, terminated, truncated, info

    def render(self, mode="human"):
 
        pygame.display.init()
        self.window = pygame.display.set_mode(
                (int(self.grid_size * self.cell_size), int(self.grid_size * self.cell_size))
            )
        pygame.display.set_caption("CustomEnv")
        if self.clock is None:
            self.clock = pygame.time.Clock()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                import sys
                sys.exit() # Exit the program

            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = event.pos
                self.hand_position = np.array([mouse_x/self.cell_size, mouse_y/self.cell_size])

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:  # 空格键切换暂停
                    self.pause = not self.pause


        canvas = pygame.Surface((self.grid_size * self.cell_size, self.grid_size * self.cell_size))
        canvas.fill(WHITE)
        virus_image = pygame.image.load("hand.png").convert_alpha()  # Load an image if needed, but not used here
        robot_image = pygame.transform.scale(virus_image, (int(self.cell_size * 2), int(self.cell_size * 2)))  # Scale the image
        # New: Draw the trajectory
        if len(self.trajectory_points) > 1:
            scaled_points = []
            for point in self.trajectory_points:
                scaled_points.append((int(point[0] * self.cell_size), int(point[1] * self.cell_size)))
            
            # Draw lines between consecutive points
            pygame.draw.lines(canvas, BLUE, False, scaled_points, 2) # Blue line, not closed, 2 pixels wide
            
            # Optionally, draw small circles at each point to emphasize
            for point_coord in scaled_points:
                pygame.draw.circle(canvas, BLUE, point_coord, 3) # Small blue circles

        # Draw robot
        pygame.draw.circle(
            canvas,
            RED,
            (int(self.robot_position[0] * self.cell_size), int(self.robot_position[1] * self.cell_size)),
            int(self.cell_size * 0.2)
        )
        # Draw obstacles

        canvas.blit(robot_image, (int((self.hand_position[0]-1) * self.cell_size), int((self.hand_position[1]-1) * self.cell_size+1)))
        pygame.draw.circle(canvas,
                            GREEN, 
                            (int((self.hand_position[0]) * self.cell_size), 
                            int((self.hand_position[1]) * self.cell_size+1)), 
        int(self.cell_size * 0.2)
        )

        self.window.blit(canvas, canvas.get_rect())
        pygame.event.pump()
        pygame.display.flip()
        self.clock.tick(self.metadata["render_fps"])
        time.sleep(0.5)

    def close(self):
        pygame.display.quit()
        pygame.quit()





In [3]:
def render_environment(robot_position, hand_position, trajectory_points, grid_size=10, cell_size=50):
    WHITE = (255, 255, 255)
    RED = (255, 0, 0)
    GREEN = (0, 255, 0)
    BLUE = (0, 0, 255)
    pygame.init()
    window = pygame.display.set_mode((grid_size * cell_size*2, grid_size * cell_size))
    canvas = pygame.Surface((grid_size * cell_size*2, grid_size * cell_size))
    canvas.fill(WHITE)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            import sys
            sys.exit() # Exit the program

        elif event.type == pygame.MOUSEBUTTONDOWN:
            mouse_x, mouse_y = event.pos
            hand_position = np.array([mouse_x/cell_size, mouse_y/cell_size])

    if len(trajectory_points) > 1:
        scaled_points = [(int(point[0] * cell_size), int(point[1] * cell_size)) for point in trajectory_points]
        pygame.draw.lines(canvas, BLUE, False, scaled_points, 2)
        for point_coord in scaled_points:
            pygame.draw.circle(canvas, BLUE, point_coord, 3)

    pygame.draw.lines(canvas, (255, 224, 189), False,[hand_position*cell_size, [grid_size*cell_size*0.75,grid_size*cell_size]],width=25)

    virus_image = pygame.image.load("hand.png").convert_alpha()  # Load an image if needed, but not used here
    robot_image = pygame.transform.scale(virus_image, (int(cell_size * 2), int(cell_size * 2)))  # Scale the 
    pygame.draw.circle(canvas, RED, (int(robot_position[0] * cell_size), int(robot_position[1] * cell_size)), int(cell_size * 0.2))
    pygame.draw.circle(canvas, GREEN, (int(hand_position[0] * cell_size), int(hand_position[1] * cell_size)), int(cell_size * 0.2))
    canvas.blit(robot_image, (int((hand_position[0]-1) * cell_size), int((hand_position[1]-1) * cell_size)))
    font = pygame.font.Font(None, 24)
    text = font.render(f"{hand_position[0]},{hand_position[1]}", True, BLUE)
    text_rect = text.get_rect()
    text_rect.center = (int(hand_position[0] * cell_size), int(hand_position[1] * cell_size))
    window.blit(canvas, canvas.get_rect())
    # window.blit(text, text_rect)
    pygame.display.flip()

    return hand_position

In [4]:
from collections import deque
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CallbackList
class DebugCallback(BaseCallback):
    def __init__(self, env, render_freq=10000, n_episodes=1, log_freq=10000, verbose=1):
        super().__init__(verbose)
        self.log_freq = log_freq
        # 用deque统计最近N个done的终止原因，避免内存爆炸
        self.termination_reasons = deque(maxlen=1000)  # 统计最近1000次终止
        self.env_to_render = env
        self.render_freq = render_freq
        self.n_episodes = n_episodes

    def _on_step(self) -> bool:
        # 先从env info里读取终止原因
        # print((self.locals.keys()))
        infos = self.locals.get('infos', None)

        dones = self.locals.get('dones', None)
        if infos is not None and dones is not None:
            for done, info in zip(dones, infos):
                if done and info is not None and 'done_reason' in info:
                    self.termination_reasons.append(info['done_reason'])

        # 每log_freq步打印信息
        # if self.num_timesteps % self.render_freq == 0 and self.verbose:
        #     for ep in range(self.n_episodes):
        #         obs = self.env_to_render.reset()
        #         done = False
        #         while not done:
        #             action, _states = self.model.predict(obs, deterministic=True)
        #             obs, rewards, done, info = self.env_to_render.step(action)
        #             self.env_to_render.render()
        #             time.sleep(0.6)

        #             if done:

        #                 self.env_to_render.close()
        #                 break





        if self.num_timesteps % self.log_freq == 0 and self.verbose:
            log = self.model.logger.name_to_value
            ep_rew = log.get('rollout/ep_rew_mean', None)
            ep_len = log.get('rollout/ep_len_mean', None)
            loss = log.get('train/loss', None)
            v_loss = log.get('train/value_loss', None)
            p_loss = log.get('train/policy_gradient_loss', None)
            ent_loss = log.get('train/entropy_loss', None)
            kl = log.get('train/approx_kl', None)

            # 统计终止原因比例
            total = len(self.termination_reasons)
            if total > 0:
                count_hand = sum(1 for r in self.termination_reasons if r == 'out of bounds')
                ratio_hand = count_hand / total
            else:
                ratio_hand = 0.0
            # print(f"[{self.num_timesteps:7d}] ep_rew_mean={ep_rew}, ep_len_mean={ep_len}, loss={loss:.3f}, "
            #       f"v_loss={v_loss:.3f}, p_loss={p_loss:.3f}, ent_loss={ent_loss:.3f}, kl={kl:.4f}, "
            #       f"termination_reason_hand_ratio={ratio_hand:.3f}")
            self.logger.record("custom/termination_reason_ratio", ratio_hand)
            self.logger.dump(step=self.num_timesteps)
            # print(f"termination_reason_hand_ratio={ratio_hand:.3f}")
        return True


In [4]:
import gymnasium as gym
import torch
from stable_baselines3 import PPO,SAC
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecMonitor
from stable_baselines3.common.monitor import Monitor


env =DummyVecEnv([lambda: Monitor(CustomEnv(render_mode='human'))])  # Wrap the environment with Monitor for logging

# env = VecNormalize(env, norm_obs=True, norm_reward=True)
policy_kwargs = dict(
    net_arch=[dict(pi=[128,256,256,128], qf=[128,256,256,128])],
    # activation_fn=torch.nn.ReLU  # 改为 ReLU，通常更适合稀疏奖励
)
policy_kwargs = dict(
    net_arch=[128,256,256,128],
    # activation_fn=torch.nn.ReLU  # 改为 ReLU，通常更适合稀疏奖励
)
# model = PPO("MlpPolicy", env, verbose=1,n_steps=4096, ent_coef=0.01,policy_kwargs=policy_kwargs,tensorboard_log="./ppo_custom_env_tensorboard/")
# model = PPO.load("logs/best_model4/best_model.zip",env=env)  # Load the best model
# model.learning_rate = 0.0008
# model.ent_coef = 0.02  # Set a lower learning rate for fine-tuning

model = SAC("MlpPolicy", env, verbose=1,ent_coef='auto',policy_kwargs=policy_kwargs,tensorboard_log="./sac_custom_env_tensorboard/")
# model = SAC.load("logs/best_model_sac10/best_model.zip",env=env)  # Load the best model

eval_callback = EvalCallback(
    env,
    best_model_save_path='./logs/best_model_sac13/',
    log_path = './logs/',
    eval_freq=10000,  # 每1000步评估一次
    deterministic=True,
    render=True,
    n_eval_episodes=10,  # 每次评估5个episode
)



debug_callback = DebugCallback(env=env,log_freq=4096, verbose=1)
callback = CallbackList([eval_callback, debug_callback])


model.learn(total_timesteps=500000,callback=callback)
# env.save("vec_env.pkl")  # Save the environment state


env.close()

KeyboardInterrupt: 

In [8]:
from stable_baselines3 import PPO,SAC

# env =DummyVecEnv([lambda: Monitor(CustomEnv())])  # Wrap the environment with Monitor for logging

# env = VecNormalize.load('vec_norm.pkl',env)
# env = VecNormalize(env)  # Apply normalization to the environment
# seed = np.random.randint(0,1000)
# from custom_env import CustomEnv
env = CustomEnv()
env.random = False



model= SAC.load("logs/best_model_sac12/best_model.zip",env=env)  # Load the best model
obs,_ = env.reset()
# obs = np.array([7.2759714 , 1.6427798 , 1.2703862 , 2.808      , 0.90284324, 0.5660193 ,
#  6.1175804 , 1.6427798 ])
# for _ in range(5):
#     print(model.predict(obs, deterministic=True))
# print(obs)
for i in range(1000000):

    # obs = env._get_obs()
    obs = env._get_obs()
    
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, teminated,_, info = env.step(action)
    print(f"obs:{obs}")
    print(f"action:{action}")



    print(info["robot_position"],info["hand_position"])
    print("Reward:", reward)
    # env.render()
    env.hand_position = render_environment(info["robot_position"], info["hand_position"], env.trajectory_points)
    time.sleep(0.6)  # Control the frame rate
    if teminated:
        env.hand_position = render_environment(info["robot_position"], info["hand_position"], env.trajectory_points)
        time.sleep(0.6)
        env.close()
      
        break
        print("Resetting environment")

AttributeError: partially initialized module 'fsspec' has no attribute 'utils' (most likely due to a circular import)